# Segmenting and Clustering Neighborhoods in Toronto

### STEP 1 - Loading data from wikipedia page

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
# Taking data from Wikipedia
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url).text

# Beautiful Soup
soup = BeautifulSoup(res, 'lxml')

In [3]:
#Finding table in wiki page
my_table = soup.find('table', {'class': 'wikitable sortable'})

In [4]:
#Obtaining the table 
rows = []
columns = []

for index, tr in enumerate(my_table.find_all('tr')):
    data = []
    for td in tr.find_all(['th','td']):
        data.append(td.text.rstrip())
    
    if (index == 0):
        columns = data
    else:
        rows.append(data)

#Converting into Pandas DataFrame
table_df = pd.DataFrame(data = rows, columns = columns)
table_df.head(10)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


### STEP 2 - Table data cleaning

In [5]:
#Removing not assigned boroughs
table_df = table_df[table_df['Borough'] != 'Not assigned']
table_df.reset_index(drop=True, inplace=True)
table_df.head(10)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
# Combine rows with the same postal code using comma separator

table_df["Neighborhood"] = table_df.groupby("Postal code")["Neighborhood"].transform(lambda x: ', '.join(x))
table_df['Neighborhood'] = table_df['Neighborhood'].str.replace('/',', ')
table_df.head(10)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
table_df['Neighborhood'].replace("Not assigned", table_df["Borough"],inplace=True)
table_df.head(10)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
table_df.shape

(103, 3)